In [3]:
%matplotlib inline
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Подход 1: градиентный бустинг "в лоб"

In [4]:
features = pd.read_csv('features.csv')
features_test = pd.read_csv('features_test.csv')

In [6]:
x_dropf = features.drop(
    ['duration', 'radiant_win', 'tower_status_radiant',
     'tower_status_dire', 'barracks_status_radiant', 
     'barracks_status_dire'], axis=1
)

full_filled_dropf = x_dropf.dropna(axis=1)

# Поля с пропусками
for item in x_dropf.drop(full_filled_dropf.columns, axis=1).columns:
    print item

first_blood_time
first_blood_team
first_blood_player1
first_blood_player2
radiant_bottle_time
radiant_courier_time
radiant_flying_courier_time
radiant_first_ward_time
dire_bottle_time
dire_courier_time
dire_flying_courier_time
dire_first_ward_time


In [11]:
# Заменяем пропуски на нули
features = features.fillna(value=0)
features_test = features_test.fillna(value=0)

In [12]:
from sklearn import cross_validation
from sklearn import ensemble

kf = cross_validation.KFold(len(features),n_folds=5,shuffle=True,random_state=241)
features_target = features['radiant_win']
features_train = features.drop(['duration','radiant_win','tower_status_radiant',
                                'tower_status_dire','barracks_status_dire',
                                'barracks_status_radiant'],axis=1)

import time
import datetime

start_time = datetime.datetime.now()

alg = ensemble.GradientBoostingClassifier(n_estimators=30, verbose=True, random_state=241,subsample=0.5)
alg.fit(features_train, features_target)
roc30 = cross_validation.cross_val_score(alg, features_train, features_target, cv=kf, n_jobs=-1,scoring='roc_auc').mean()

print "roc-auc 30 trees: ", roc30

print 'Time tree:', datetime.datetime.now() - start_time

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           1.3787           0.0062            1.37m
         2           1.3733           0.0053            1.25m
         3           1.3683           0.0048            1.19m
         4           1.3629           0.0042            1.14m
         5           1.3579           0.0040            1.07m
         6           1.3542           0.0043            1.01m
         7           1.3489           0.0038           57.41s
         8           1.3459           0.0043           55.92s
         9           1.3413           0.0037           52.91s
        10           1.3365           0.0027           50.33s
        20           1.3087           0.0019           24.40s
        30           1.2879           0.0014            0.00s
roc-auc 30 trees:  0.690710875095
Time tree: 0:04:17.307000


In [13]:
start_time = datetime.datetime.now()

alg40 = ensemble.GradientBoostingClassifier(n_estimators=40, verbose=True, random_state=241,subsample=0.5)
alg40.fit(features_train, features_target)
roc40 = cross_validation.cross_val_score(alg40, features_train, features_target, cv=kf, n_jobs=-1,scoring='roc_auc').mean()
print "roc-auc 40 trees: ", roc40

print 'Time tree:', datetime.datetime.now() - start_time

print "roc-auc delta:",  roc40-roc30

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           1.3787           0.0062            1.65m
         2           1.3733           0.0053            1.66m
         3           1.3683           0.0048            1.54m
         4           1.3629           0.0042            1.49m
         5           1.3579           0.0040            1.44m
         6           1.3542           0.0043            1.39m
         7           1.3489           0.0038            1.36m
         8           1.3459           0.0043            1.31m
         9           1.3413           0.0037            1.26m
        10           1.3365           0.0027            1.22m
        20           1.3087           0.0019           48.28s
        30           1.2879           0.0014           24.18s
        40           1.2755           0.0010            0.00s
roc-auc 40 trees:  0.69545547381
Time tree: 0:05:36.290000
roc-auc delta: 0.00474459871544


## Ответы по первой части
1. Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?
first_blood_time
first_blood_team
first_blood_player1
first_blood_player2
radiant_bottle_time
radiant_courier_time
radiant_flying_courier_time
radiant_first_ward_time
dire_bottle_time
dire_courier_time
dire_flying_courier_time
dire_first_ward_time

first_blood_time - ни одна из команд за первые 5 минут не совершила первой крови
first_blood_team - см. выше, ни одна из команд не соверщила первой крови.

2. Как называется столбец, содержащий целевую переменную?
Целевая переменная в radiant win

3. Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Какое качество при этом получилось?
Время выполения - 0:04:17
Качество (в качестве метрики используем ROC-AUC) - 0.690710875095

4. Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что можно сделать, чтобы ускорить его обучение при увеличении количества деревьев?
Проведён эесперемент на 40 деревьях - при этом качество увеличелос на 0.0048.
Не считаю целесообразным увеличивать количетво деревьев, т.к. время построения 40 деревьев больше на минуту.

# Подход 2: логистическая регрессия

In [13]:
import sklearn.cross_validation
import sklearn.ensemble
import sklearn.metrics
import sklearn.preprocessing

X = features.ix[:,:-6]
y = features['radiant_win']

XX = sklearn.preprocessing.StandardScaler().fit_transform(X)

fold = sklearn.cross_validation.KFold(n=len(XX), n_folds=5, random_state=1, shuffle=True)
clf = sklearn.linear_model.LogisticRegression()
grid = {'C': np.linspace(0.001,1,10)}
gs = sklearn.grid_search.GridSearchCV(clf, grid, scoring='roc_auc', cv=fold, n_jobs=8)
%time gs.fit(XX, y)
for score in gs.grid_scores_:
    print("Quality {} for {}".format(score.mean_validation_score, score.parameters))
print("Best param: ", gs.best_params_)

Wall time: 3min 2s
Quality 0.71627623552 for {'C': 0.001}
Quality 0.716421655412 for {'C': 0.112}
Quality 0.716419848882 for {'C': 0.223}
Quality 0.716419358408 for {'C': 0.33400000000000002}
Quality 0.716418795932 for {'C': 0.44500000000000001}
Quality 0.716418521695 for {'C': 0.55600000000000005}
Quality 0.716418303507 for {'C': 0.66700000000000004}
Quality 0.716418417868 for {'C': 0.77800000000000002}
Quality 0.716418212351 for {'C': 0.88900000000000001}
Quality 0.71641818587 for {'C': 1.0}
('Best param: ', {'C': 0.112})


Наилучшее качетство при С=0.112, оно равно 0.716421655412. Данное значение привосходит результат при градиентном бустинге. 
Лоигистичесткая регрессия позволяет найти гиперплоскость максимально разделяющую объекты двух классов - она лучше работает когда объекты слабо сгруппированы в пространстве.

In [14]:
def del_from_x(X):
    drops = ["{}{}_hero".format(c, i) for c in 'rd' for i in range(1,6)]
    X2 = X.drop('lobby_type', axis=1).drop(drops, axis=1)
    return X2

X2 = del_from_x(X)
X2.head()

,match_id,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
0,0,1430198770,5,2098,1489,20,0,0,7,3,...,0,35,103,-84,221,3,4,2,2,-52
1,1,1430220345,4,1188,1033,9,0,1,12,4,...,0,-20,149,-84,195,5,4,3,1,-5
2,2,1430227081,4,1319,1270,22,0,0,12,3,...,1,-39,45,-77,221,3,4,3,1,13
3,3,1430263531,4,1779,1056,14,0,0,5,2,...,0,-30,124,-80,184,0,4,2,0,27
4,4,1430282290,4,1431,1090,8,1,0,8,2,...,0,46,182,-80,225,6,3,3,0,-16


In [15]:
import sklearn.cross_validation
import sklearn.ensemble
import sklearn.metrics
import sklearn.preprocessing

XX2 = sklearn.preprocessing.StandardScaler().fit_transform(X2)

fold = sklearn.cross_validation.KFold(n=len(XX2), n_folds=5, random_state=1, shuffle=True)
clf = sklearn.linear_model.LogisticRegression()
grid = {'C': np.linspace(0.001,1,10)}
gs = sklearn.grid_search.GridSearchCV(clf, grid, scoring='roc_auc', cv=fold, n_jobs=8)
%time gs.fit(XX2, y)
for score in gs.grid_scores_:
    print("Quality {} for {}".format(score.mean_validation_score, score.parameters))
print("Best param: ", gs.best_params_)

Wall time: 2min 46s
Quality 0.716311210036 for {'C': 0.001}
Quality 0.716464409127 for {'C': 0.112}
Quality 0.716462877056 for {'C': 0.223}
Quality 0.716462348693 for {'C': 0.33400000000000002}
Quality 0.716462204764 for {'C': 0.44500000000000001}
Quality 0.716461871245 for {'C': 0.55600000000000005}
Quality 0.71646181819 for {'C': 0.66700000000000004}
Quality 0.716461576676 for {'C': 0.77800000000000002}
Quality 0.716461501546 for {'C': 0.88900000000000001}
Quality 0.716461422106 for {'C': 1.0}
('Best param: ', {'C': 0.112})


Качество результатов практические не изменилось, т.к. линейная модель плохо работает с категориальными признаками. Исключение данных не повлияло на выборку.

In [23]:
unique_heroes = []
for name in ['r1_hero','r2_hero','r3_hero','r4_hero','r5_hero','d1_hero','d2_hero','d3_hero','d4_hero','d5_hero']:
    for u in pd.unique(features_test[name]):
        if u not in unique_heroes:
            unique_heroes.append(u)
len_unique_heroes = len(unique_heroes)
print 'количество уникальных героев в игре:', len_unique_heroes

количество уникальных героев в игре: 108


In [26]:
def pick_from(X):
    X_pick = np.zeros((X.shape[0], len_unique_heroes))
    for i, match_id in enumerate(X.index):
        for p in range(1,6):
            X_pick[i, np.where(unique_heroes == X.ix[match_id, 'r%d_hero' % p])] = 1
            X_pick[i, np.where(unique_heroes == X.ix[match_id, 'd%d_hero' % p])] = -1
    return X_pick

X_pick = pick_from(X)
X_pick

array([[ 0.,  1.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1., -1., ...,  0.,  0.,  0.]])

In [28]:
X4 = X2.join(pd.DataFrame(X_pick, index=features.index))
X4.head()

,match_id,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,...,98,99,100,101,102,103,104,105,106,107
0,0,1430198770,5,2098,1489,20,0,0,7,3,...,0,0,0,0,0,0,0,1,0,0
1,1,1430220345,4,1188,1033,9,0,1,12,4,...,0,0,0,0,0,0,0,0,0,0
2,2,1430227081,4,1319,1270,22,0,0,12,3,...,0,0,0,0,0,0,0,0,0,0
3,3,1430263531,4,1779,1056,14,0,0,5,2,...,0,0,0,0,0,0,0,0,0,0
4,4,1430282290,4,1431,1090,8,1,0,8,2,...,0,0,0,-1,0,0,0,0,0,0


In [29]:
XX4 = sklearn.preprocessing.StandardScaler().fit_transform(X4)

fold = sklearn.cross_validation.KFold(n=len(XX4), n_folds=5, random_state=1, shuffle=True)
clf = sklearn.linear_model.LogisticRegression()
grid = {'C': np.linspace(0.001,1,10)}
gs = sklearn.grid_search.GridSearchCV(clf, grid, scoring='roc_auc', cv=fold, n_jobs=8)
%time gs.fit(XX4, y)
for score in gs.grid_scores_:
    print("Quality {} for {}".format(score.mean_validation_score, score.parameters))
print("Best param: ", gs.best_params_)

Wall time: 5min 21s
Quality 0.751649175002 for {'C': 0.001}
Quality 0.751901751325 for {'C': 0.112}
Quality 0.751898168114 for {'C': 0.223}
Quality 0.751896478549 for {'C': 0.33400000000000002}
Quality 0.751895795275 for {'C': 0.44500000000000001}
Quality 0.751895467936 for {'C': 0.55600000000000005}
Quality 0.751895352368 for {'C': 0.66700000000000004}
Quality 0.751895209351 for {'C': 0.77800000000000002}
Quality 0.75189532054 for {'C': 0.88900000000000001}
Quality 0.751895163809 for {'C': 1.0}
('Best param: ', {'C': 0.112})


Качество возросло до 0.751901751325. Поскольку значения классификатора до этого момента перечисленные в задании категориальные признаки не вносили значительного вклада при построении класификатора ввиду особенностей класификатора. Теперь же эти данные адаптированы для использования классификатором.

In [37]:
features_test = pd.read_csv('./features_test.csv')
X_test = features_test
X_test = X_test.fillna(0)
X2_test = del_from_x(X_test)
X4_test = X2_test.join(pd.DataFrame(pick_from(X_test), index=features_test.index))

In [38]:
X4_test.head()

,match_id,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,...,98,99,100,101,102,103,104,105,106,107
0,6,1430287923,4,1103,1089,8,0,1,9,3,...,0,0,0,0,-1,0,0,0,0,0
1,7,1430293357,2,556,570,1,0,0,9,4,...,0,0,0,0,0,0,0,0,0,0
2,10,1430301774,2,751,808,1,0,0,13,2,...,0,-1,0,0,0,0,0,0,0,0
3,13,1430323933,3,708,903,1,1,1,11,2,...,-1,0,0,0,0,0,0,0,0,0
4,16,1430331112,4,1259,661,4,0,0,9,5,...,0,0,0,0,0,0,0,0,0,0


In [39]:
XX4 = sklearn.preprocessing.StandardScaler().fit_transform(X4)
logreg = sklearn.linear_model.LogisticRegression(C=gs.best_params_['C'])
logreg.fit(XX4, y)

XX4_test = sklearn.preprocessing.StandardScaler().fit_transform(X4_test)
res = logreg.predict_proba(XX4_test)
print(res.min(), res.max())
print(res)

(0.0034224716484642093, 0.99657752835153579)
[[ 0.16771354  0.83228646]
 [ 0.23547667  0.76452333]
 [ 0.80716253  0.19283747]
 ..., 
 [ 0.75725084  0.24274916]
 [ 0.3570561   0.6429439 ]
 [ 0.56060595  0.43939405]]


Результат выглядит адекватным. Он находится на отрезке [0, 1], не совпадают. Минимальное значение 0.0034224716484642093 максимальне 0.99657752835153579

## Отчёт по второй части
1. Какое качество получилось у логистической регрессии над всеми исходными признаками? Как оно соотносится с качеством градиентного бустинга? Чем можно объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?
Наилучшее качетство при С=0.112, оно равно 0.716421655412. Данное значение привосходит результат при градиентном бустинге. 
Лоигистичесткая регрессия позволяет найти гиперплоскость максимально разделяющую объекты двух классов - она лучше работает когда объекты слабо сгруппированы в пространстве.

2. Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? Чем можно объяснить это изменение?
Качество результатов практические не изменилось, т.к. линейная модель плохо работает с категориальными признаками. Исключение данных не повлияло на выборку.

3. Сколько различных идентификаторов героев существует в данной игре?
Количество уникальных идентификаторов героев в игре: 108

4. Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем можно это объяснить?
Качество возросло до 0.751901751325. Поскольку значения классификатора до этого момента перечисленные в задании категориальные признаки не вносили значительного вклада при построении класификатора ввиду особенностей класификатора. Теперь же эти данные адаптированы для использования классификатором.

5. Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?
Минимальное значение 0.0034224716484642093 максимальне 0.99657752835153579